In [ ]:
!nvidia-smi

Fri Jul  8 10:34:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install torchensemble

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Function
from torchvision import models
from torchvision import utils
from matplotlib import pyplot as plt
from torchvision import datasets, models, transforms
from torch.utils.data.dataloader import default_collate

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
#Create the dataset loader
input_path = 'drive/MyDrive/cxr_splitdata/'

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((256,256)),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'val':
    transforms.Compose([
        transforms.Resize((256,256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((256,256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize
    ])
}

In [ ]:
image_datasets = {
    'train': 
    datasets.ImageFolder(input_path + 'train', data_transforms['train']),
    'val': 
    datasets.ImageFolder(input_path + 'val', data_transforms['val']),  
    'test': 
    datasets.ImageFolder(input_path + 'test', data_transforms['test'])
}


In [ ]:
dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True,
                                #collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)),
                                num_workers=0),
    'val':
    torch.utils.data.DataLoader(image_datasets['val'],
                                batch_size=32,
                                shuffle=False,
                                #collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)),
                                num_workers=0), 
    
    'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=32,
                                shuffle=False,
                                #collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)),
                                num_workers=0)
}

In [ ]:
#MANUAL VGG IMPLEMENTED MODEL
VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M",],
    "VGG19": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M",],
}


class VGG_net(nn.Module):
    def __init__(self, in_channels=3, num_classes=2):
        super(VGG_net, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG_types["VGG11"])

        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=(3, 3),
                        stride=(1, 1),
                        padding=(1, 1),
                    ),
                    nn.BatchNorm2d(x),
                    nn.ReLU(),
                ]
                in_channels = x
            elif x == "M":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)


In [ ]:
#in_channels = 3
#num_classes = 2
#base_estimator = CNN(in_channels=in_channels, num_classes=num_classes)

In [ ]:
in_channels = 3
num_classes = 2
base_estimator = VGG_net(in_channels=3, num_classes=2)
print(base_estimator)

VGG_net(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): MaxPool2d(kernel_size=(2, 2), str

In [ ]:
# Load data
train_loader = dataloaders['train']
val_loader = dataloaders['val']
test_loader = dataloaders['test']

In [ ]:
learning_rate = 0.003
weight_decay = 0.3
epochs = 5

In [ ]:
from torchensemble import VotingClassifier

In [ ]:
# Define the ensemble
ensemble = VotingClassifier(
    estimator=base_estimator,              # here is your deep learning model
    n_estimators=5,                        # number of base estimators
)

In [ ]:
# Set the criterion
criterion = nn.CrossEntropyLoss()           # training objective
ensemble.set_criterion(criterion)

In [ ]:
# Set the optimizer
ensemble.set_optimizer(
    "Adam",                                 # type of parameter optimizer
    lr=learning_rate,                       # learning rate of parameter optimizer
    weight_decay=weight_decay              # weight decay of parameter optimizer
)


In [ ]:
# Set the learning rate scheduler
ensemble.set_scheduler(
    "CosineAnnealingLR",                    # type of learning rate scheduler
    T_max=epochs                           # additional arguments on the scheduler
)

In [ ]:
# Train the ensemble
ensemble.fit(
    train_loader,
    epochs=epochs                          # number of training epochs
)

Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 0.72659 | Correct: 15/32
Estimator: 000 | Epoch: 000 | Batch: 100 | Loss: 1.16479 | Correct: 20/32
Estimator: 001 | Epoch: 000 | Batch: 000 | Loss: 0.71479 | Correct: 13/32
Estimator: 001 | Epoch: 000 | Batch: 100 | Loss: 1.15906 | Correct: 22/32
Estimator: 002 | Epoch: 000 | Batch: 000 | Loss: 0.69824 | Correct: 19/32
Estimator: 002 | Epoch: 000 | Batch: 100 | Loss: 1.17345 | Correct: 18/32
Estimator: 003 | Epoch: 000 | Batch: 000 | Loss: 0.67149 | Correct: 17/32
Estimator: 003 | Epoch: 000 | Batch: 100 | Loss: 0.74384 | Correct: 17/32
Estimator: 004 | Epoch: 000 | Batch: 000 | Loss: 0.73685 | Correct: 13/32
Estimator: 004 | Epoch: 000 | Batch: 100 | Loss: 4.83019 | Correct: 11/32
Estimator: 000 | Epoch: 001 | Batch: 000 | Loss: 0.78310 | Correct: 16/32
Estimator: 000 | Epoch: 001 | Batch: 100 | Loss: 0.60630 | Correct: 20/32
Estimator: 001 | Epoch: 001 | Batch: 000 | Loss: 0.95625 | Correct: 16/32
Estimator: 001 | Epoch: 001 | Batch: 1

In [ ]:
# Evaluate the ensemble
acc = ensemble.evaluate(val_loader)         # validation accuracy

In [ ]:
print(acc)

50.0


In [ ]:
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torchvision

class DensNet(nn.Module):
    def __init__(self, num_classes=2, num_channels=3):
        super().__init__()
        preloaded = torchvision.models.densenet121(pretrained=True)
        self.features = preloaded.features
        self.features.conv0 = nn.Conv2d(num_channels, 64, 7, 2, 3)
        self.classifier = nn.Linear(1024, num_classes, bias=True)
        del preloaded
        
    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1)).view(features.size(0), -1)
        out = self.classifier(out)
        return out

In [ ]:
in_channels = 3
num_classes = 2
base_estimator2 = DensNet(num_classes=num_classes, num_channels=in_channels)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

In [ ]:
learning_rate = 0.003
weight_decay = 0.3
epochs = 5

In [ ]:
from torchensemble import VotingClassifier

In [ ]:
# Define the ensemble
ensemble = VotingClassifier(
    estimator=base_estimator2,              # here is your deep learning model
    n_estimators=5,                        # number of base estimators
)

In [ ]:
# Set the criterion
criterion = nn.CrossEntropyLoss()           # training objective
ensemble.set_criterion(criterion)

In [ ]:
# Set the optimizer
ensemble.set_optimizer(
    "Adam",                                 # type of parameter optimizer
    lr=learning_rate,                       # learning rate of parameter optimizer
    weight_decay=weight_decay              # weight decay of parameter optimizer
)


In [ ]:
# Set the learning rate scheduler
ensemble.set_scheduler(
    "CosineAnnealingLR",                    # type of learning rate scheduler
    T_max=epochs                           # additional arguments on the scheduler
)

In [ ]:
# Train the ensemble
ensemble.fit(
    train_loader,
    log_interval=10,
    test_loader=val_loader,
    epochs=epochs                          # number of training epochs
)

Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 0.75224 | Correct: 14/32
Estimator: 000 | Epoch: 000 | Batch: 010 | Loss: 0.53421 | Correct: 27/32
Estimator: 000 | Epoch: 000 | Batch: 020 | Loss: 0.80177 | Correct: 15/32
Estimator: 000 | Epoch: 000 | Batch: 030 | Loss: 0.66552 | Correct: 25/32
Estimator: 000 | Epoch: 000 | Batch: 040 | Loss: 0.59527 | Correct: 21/32
Estimator: 000 | Epoch: 000 | Batch: 050 | Loss: 0.60640 | Correct: 22/32
Estimator: 000 | Epoch: 000 | Batch: 060 | Loss: 0.54923 | Correct: 28/32
Estimator: 000 | Epoch: 000 | Batch: 070 | Loss: 0.59878 | Correct: 21/32
Estimator: 000 | Epoch: 000 | Batch: 080 | Loss: 0.65911 | Correct: 19/32
Estimator: 000 | Epoch: 000 | Batch: 090 | Loss: 0.55955 | Correct: 22/32
Estimator: 000 | Epoch: 000 | Batch: 100 | Loss: 0.49618 | Correct: 25/32
Estimator: 000 | Epoch: 000 | Batch: 110 | Loss: 0.57862 | Correct: 21/32
Estimator: 000 | Epoch: 000 | Batch: 120 | Loss: 0.58071 | Correct: 23/32
Estimator: 000 | Epoch: 000 | Batch: 1

In [ ]:
# Evaluate the ensemble
acc = ensemble.evaluate(test_loader)         # validation accuracy

In [ ]:
print(acc)

78.1718963165075
